#### ver4 : 유사 값 분류를 통한 K-means 진행 

- columns : 외부 온도, 차량 속도, 일사량, 습도
- xlsx : clustered_results_4columns_k_2.xlsx, cross_tab_results

In [16]:
import glob
import pandas as pd
import numpy as np

In [17]:
csv_path = "/home/miserver10/workspace_vscode/datasets/data/*.csv"

csv_files = glob.glob(csv_path)

df_list = [pd.read_csv(csv_file) for csv_file in csv_files]
df = pd.concat(df_list, ignore_index=True)  #ignore_index = 기존 인덱스 무시하여 새로운 인덱스 생성(연속으로 이어진다)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 54 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Front_Seat_LH_Temp Unit: Degree ('c)     339932 non-null  float64
 1   Front_Seat_RH_Temp Unit: Degree('c)      339932 non-null  float64
 2   Rear_Seat_CT_Temp Unit: Degree('c)       339932 non-null  float64
 3   Front_Roof_CT_Temp Unit: Degree('c)      339932 non-null  float64
 4   Front_Roof_LH_Temp Unit: Degree('c)      339932 non-null  float64
 5   Front_Roof_RH_Temp Unit: Degree('c)      339932 non-null  float64
 6   Rear_Roof_LH_Temp Unit: Degree('c)       339932 non-null  float64
 7   Rear_Roof_RH_Temp Unit: Degree('c)       339932 non-null  float64
 8   DRV_LH_Vent_Wind_Velocity Unit: m / s    339932 non-null  float64
 9   DRV_RH_Vent_Wind_Velocity Unit: m / s    339932 non-null  float64
 10  DRV_Foot_Vent_Wind_Velocity Unit

In [18]:
drop_list = ["Front_Seat_LH_Temp Unit: Degree ('c)",
       "Front_Seat_RH_Temp Unit: Degree('c)",
       "Rear_Seat_CT_Temp Unit: Degree('c)",
       "Front_Roof_CT_Temp Unit: Degree('c)",
       "Front_Roof_LH_Temp Unit: Degree('c)",
       "Front_Roof_RH_Temp Unit: Degree('c)",
       "Rear_Roof_LH_Temp Unit: Degree('c)",
       "Rear_Roof_RH_Temp Unit: Degree('c)",
       "DRV_LH_Vent_Wind_Velocity Unit: m / s",
       "DRV_RH_Vent_Wind_Velocity Unit: m / s",
       "DRV_Foot_Vent_Wind_Velocity Unit: m / s",
       "DRV_Seat_Vent_Wind_Velocity Unit: m / s",
       "PAS_LH_Vent_Wind_Velocity Unit: m / s",
       "PAS_RH_Vent_Wind_Velocity Unit: m / s",
       "PAS_Foot_Vent_Wind_Velocity Unit: m / s",
       "PAS_Seat_Vent_Wind_Velocity Unit: m / s", "5V_ADC_CH1 Unit: V",
       "5V_ADC_CH2 Unit: V", "5V_ADC_CH3 Unit: V", "5V_ADC_CH4 Unit: V",
       "5V_ADC_CH5 Unit: V", "PWM_CH8 Unit: PWM", "Frequency_CH8 Unit: Hz",
       "CR_Datc_DrModeAct","CR_Datc_TempAct", "CR_Datc_IntakeAct", "CR_Datc_AdsAct",
       "CR_Datc_DrVentTemp_C", "CR_Datc_DrFloorTemp_C", 
       "CR_Datc_ClimatePowSum_W", "LAT_ACCEL_m/s^2", "LONG_ACCEL_m/s^2",
       "YAW_RATE_'/s", "WHL_SPD_FL_km/h", "WHL_SPD_FR_km/h", "WHL_SPD_RL_km/h",
       "WHL_SPD_RR_km/h", 'CR_Datc_AcnCompSpdFB_rpm', 'CR_Datc_AptVal_PSI', 'CR_Datc_OutTemp_C', 
       'CR_Datc_OutTempC' ,'CR_Datc_EvapSnsTemp_C', 'CR_Datc_DrTempDispC', 'DATC_ModeDisp']

df = df.drop(drop_list, axis=1)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CR_Datc_AcnCompPwrCon_W  339932 non-null  int64  
 1   CR_Datc_OutTempSnr_C     339932 non-null  float64
 2   CR_Datc_PTCPwrCon_W      339932 non-null  int64  
 3   CF_Datc_Heatpump_Mode    339932 non-null  int64  
 4   CR_Datc_SetTemp_C        339932 non-null  float64
 5   CR_Datc_IncarSnsTemp_C   339932 non-null  float64
 6   DATC_BlowerLevel         339932 non-null  int64  
 7   CR_Datc_PhotoSnsFB_V     339932 non-null  float64
 8   CR_Datc_Humidity         339932 non-null  float64
 9   CR_Mcu_VehSpd_Kph_sum    339932 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 25.9 MB


In [20]:
df.rename(columns={'CR_Datc_OutTempSnr_C': '외부 온도', 'CR_Datc_AcnCompPwrCon_W': '에어컨 압축기 전력 소비량',
                   'CR_Datc_PTCPwrCon_W': '히터의 전력 소비량', 'DATC_BlowerLevel': '바람세기', 'CF_Datc_Heatpump_Mode' : '히트 펌프',
                   'CR_Datc_SetTemp_C': '설정 온도', 'CR_Datc_IncarSnsTemp_C': '내부 온도', 
                   'CR_Datc_PhotoSnsFB_V' : '일사량', 'CR_Datc_Humidity' : '습도', 'CR_Mcu_VehSpd_Kph_sum' : '차량 속도'}, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   에어컨 압축기 전력 소비량  339932 non-null  int64  
 1   외부 온도           339932 non-null  float64
 2   히터의 전력 소비량      339932 non-null  int64  
 3   히트 펌프           339932 non-null  int64  
 4   설정 온도           339932 non-null  float64
 5   내부 온도           339932 non-null  float64
 6   바람세기            339932 non-null  int64  
 7   일사량             339932 non-null  float64
 8   습도              339932 non-null  float64
 9   차량 속도           339932 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 25.9 MB


In [22]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
에어컨 압축기 전력 소비량,339932.0,253.707359,299.143430,0.0,0.000000,200.000000,420.000000,2010.000000
외부 온도,339932.0,18.767186,10.624092,-9.0,11.000000,20.500000,26.500000,39.000000
히터의 전력 소비량,339932.0,375.223221,734.062488,0.0,0.000000,0.000000,230.000000,5890.000000
히트 펌프,339932.0,1.443548,2.550282,0.0,0.000000,0.000000,0.000000,7.000000
설정 온도,339932.0,21.260543,1.945197,0.0,20.000000,21.000000,22.500000,27.000000
내부 온도,339932.0,23.957999,2.936478,0.0,22.500000,24.000000,26.000000,35.500000
바람세기,339932.0,3.372416,1.334061,0.0,3.000000,4.000000,4.000000,9.000000
일사량,339932.0,0.293877,0.228326,0.0,0.100000,0.200000,0.440000,1.240000
습도,339932.0,43.334913,15.393059,0.0,32.000000,42.500000,53.500000,90.000000
차량 속도,339932.0,54.541180,37.157516,0.0,18.138672,63.289062,85.609375,169.335938


In [23]:
# 외부 온도: 4도씩 범주화
bins_temp = np.arange(-9, 41, 4)
labels_temp = [f'{i}-{i+4}' for i in bins_temp[:-1]]
df['외부온도범위'] = pd.cut(df['외부 온도'], bins=bins_temp, labels=labels_temp, right=False, include_lowest=True)

# 차량 속도: 50km/h씩 범주화
bins_speed = [0, 50, 100, float('inf')]  # 0-50, 50-100, 100 이상
labels_speed = ['0-50', '50-100', '100+']  # 레이블 설정
df['차량속도범위'] = pd.cut(df['차량 속도'], bins=bins_speed, labels=labels_speed, right=False, include_lowest=True)

# 일사량: 0.4씩 범주화
bins_solar = np.arange(0, 1.24 + 0.4, 0.4)
labels_solar = [f'{i:.1f}-{i+0.4:.1f}' for i in bins_solar[:-1]]
df['일사량범위'] = pd.cut(df['일사량'], bins=bins_solar, labels=labels_solar, right=False, include_lowest=True)

# 습도: 20씩 범주화
bins_humidity = np.arange(0, 91, 20)
labels_humidity = [f'{i}-{i+20}' for i in bins_humidity[:-1]]
df['습도범위'] = pd.cut(df['습도'], bins=bins_humidity, labels=labels_humidity, right=False, include_lowest=True)


In [24]:
# 결과 출력
print(df.head())

   에어컨 압축기 전력 소비량  외부 온도  히터의 전력 소비량  히트 펌프  설정 온도  내부 온도  바람세기   일사량    습도  \
0             420   22.5           0      0   18.0   27.5     4  0.48  30.0   
1             420   22.5           0      0   18.0   27.5     4  0.48  30.0   
2             420   22.5           0      0   18.0   27.5     4  0.48  30.0   
3             420   22.5           0      0   18.0   27.5     4  0.46  30.0   
4             420   22.5           0      0   18.0   27.5     4  0.46  30.0   

   차량 속도 외부온도범위 차량속도범위    일사량범위   습도범위  
0    0.0  19-23   0-50  0.4-0.8  20-40  
1    0.0  19-23   0-50  0.4-0.8  20-40  
2    0.0  19-23   0-50  0.4-0.8  20-40  
3    0.0  19-23   0-50  0.4-0.8  20-40  
4    0.0  19-23   0-50  0.4-0.8  20-40  


In [25]:

cross_tab = pd.crosstab(index=[df['외부온도범위'], df['차량속도범위'], df['일사량범위']], columns=df['습도범위'])
# 결과 출력
print(cross_tab)

습도범위                   0-20  20-40  40-60  60-80
외부온도범위 차량속도범위 일사량범위                             
-9--5  0-50   0.0-0.4    69   1443    401     22
              0.4-0.8   132    285     64      0
       50-100 0.0-0.4     0    951   1099   1140
              0.4-0.8     0    903    231    112
       100+   0.0-0.4     0    205    295    216
...                     ...    ...    ...    ...
35-39  0-50   0.8-1.2  1491      5      0      0
       50-100 0.0-0.4   205    121      0      0
              0.4-0.8  2229    660      0      0
       100+   0.0-0.4     0     21      0      0
              0.4-0.8   114     24      0      0

[93 rows x 4 columns]


In [26]:
excel_filename = 'cross_tab_results.xlsx'
cross_tab.to_excel(excel_filename)

print(f"교차 테이블 결과가 '{excel_filename}' 파일로 저장되었습니다.")

교차 테이블 결과가 'cross_tab_results.xlsx' 파일로 저장되었습니다.


In [27]:
# 카테고리별 데이터 저장을 위한 딕셔너리 생성
category_data = {}

# 모든 카테고리 조합에 대해 데이터프레임을 생성
for temp_range in df['외부온도범위'].unique():
    for speed_range in df['차량속도범위'].unique():
        for solar_range in df['일사량범위'].unique():
            for humidity_range in df['습도범위'].unique():
                # 각 카테고리 조합에 해당하는 데이터프레임 생성
                subset = df[
                (df['외부온도범위'] == temp_range) &
                (df['차량속도범위'] == speed_range) &
                (df['일사량범위'] == solar_range) &
                (df['습도범위'] == humidity_range)]
                subset = subset.drop(['외부온도범위', '차량속도범위','일사량범위','습도범위'], axis=1)  # 카테고리 이름 열 제외
                if len(subset) >= 30 :  # 데이터가 있는지 확인
                    category_data[f'tp_{temp_range}_sp_{speed_range}_so_{solar_range}_hu_{humidity_range}'] = subset

# 딕셔너리에 저장된 데이터프레임 출력
for key, value in category_data.items():
    print(f"카테고리: {key}")
    print(value)
    print()  

카테고리: tp_19-23_sp_0-50_so_0.4-0.8_hu_20-40
        에어컨 압축기 전력 소비량  외부 온도  히터의 전력 소비량  히트 펌프  설정 온도  내부 온도  바람세기   일사량  \
0                  420   22.5           0      0   18.0   27.5     4  0.48   
1                  420   22.5           0      0   18.0   27.5     4  0.48   
2                  420   22.5           0      0   18.0   27.5     4  0.48   
3                  420   22.5           0      0   18.0   27.5     4  0.46   
4                  420   22.5           0      0   18.0   27.5     4  0.46   
...                ...    ...         ...    ...    ...    ...   ...   ...   
311988             230   20.5           0      0   22.0   24.0     4  0.48   
311989             230   20.5           0      0   22.0   24.0     4  0.48   
311990             230   20.5           0      0   22.0   24.0     4  0.48   
311991             230   21.0           0      0   22.0   24.0     4  0.48   
311992             230   21.0           0      0   22.0   24.0     4  0.48   

          습도  차량 속도 

In [28]:
print(f"category_data의 크기: {len(category_data)}")

category_data의 크기: 190


In [29]:
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# # 스케일러 정의
# scaler = StandardScaler()

# # 각 카테고리별로 KMeans를 적용하고 결과를 저장할 딕셔너리 생성
# clustered_data = {}

# for key, value in category_data.items():
#     # 선택한 특성에 대해 스케일링 수행
#     scaled_value = value.copy()  # 원본 데이터 복사
#     scaled_value = scaler.fit_transform(scaled_value)
    
#     # KMeans 적용
#     kmeans = KMeans(n_clusters=3, random_state=42)
#     clusters = kmeans.fit_predict(scaled_value)
    
#     # 클러스터별 평균 계산 (스케일링 되기 전의 값으로)
#     clustered_value = value.copy()
#     clustered_value['클러스터'] = clusters
#     cluster_means = clustered_value.groupby('클러스터').mean()
    
#     # 결과 저장
#     clustered_data[key] = cluster_means

# # 클러스터링 결과를 CSV 파일로 저장
# for key, value in clustered_data.items():
#     value.to_csv(f'clustered_results_{key}.csv')
#     print(f"카테고리 '{key}'에 대한 클러스터링 결과가 'clustered_results_{key}.csv' 파일로 저장되었습니다.")

In [30]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 스케일러 정의
scaler = StandardScaler()

# 결과를 저장할 엑셀 파일 이름
excel_file = 'clustering_4columns_k_3.xlsx'

# 엑셀 파일 생성
with pd.ExcelWriter(excel_file) as writer:
    for key, value in category_data.items():
        # 외부 온도, 차량 속도, 일사량, 습도 열만 선택하여 클러스터링 수행
        features = ['외부 온도', '차량 속도', '일사량', '습도']
        selected_features = value[features]
        scaled_value = scaler.fit_transform(selected_features)

        # KMeans 적용
        kmeans = KMeans(n_clusters=3, random_state=42)
        clusters = kmeans.fit_predict(scaled_value)

        # 클러스터링 결과를 데이터프레임에 추가
        clustered_value = value.copy()
        clustered_value['클러스터'] = clusters

        # 클러스터별 평균 계산 (스케일링 되기 전의 값으로)
        cluster_means = clustered_value.groupby('클러스터').mean()

        # 결과를 엑셀 파일의 시트로 저장
        cluster_means.to_excel(writer, sheet_name=key)

# 엑셀 파일 저장 완료 메시지 출력
print(f"클러스터링 결과가 '{excel_file}' 파일에 저장되었습니다.")

/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/openpyxl/workbook/child

클러스터링 결과가 'clustering_4columns_k_3.xlsx' 파일에 저장되었습니다.
